In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import argparse
import pprint
import time
import datetime
import sys
import os
import torch
import numpy as np
import shutil
import os
import warnings
from tqdm import tqdm
import codecs
import webdataset as wds
import omegaconf

from sdofm.utils import days_hours_mins_secs_str
from sdofm.datasets import TimestampedSDOMLDataModule, SDOMLDataModule
from sdofm.pretraining import NVAE, MAE

In [26]:
cfg = omegaconf.OmegaConf.load("../experiments/pretrain_32.2M_mae_tpu_2048.yaml")

data_module = TimestampedSDOMLDataModule(
    # hmi_path=os.path.join(
    #     cfg.data.sdoml.base_directory, cfg.data.sdoml.sub_directory.hmi
    # ),
    hmi_path=None,
    aia_path=os.path.join(
        cfg.data.sdoml.base_directory, cfg.data.sdoml.sub_directory.aia
    ),
    eve_path=None,
    components=cfg.data.sdoml.components,
    wavelengths=cfg.data.sdoml.wavelengths,
    ions=cfg.data.sdoml.ions,
    frequency=cfg.data.sdoml.frequency,
    batch_size=cfg.model.opt.batch_size,
    num_workers=cfg.data.num_workers,
    val_months=[],  # cfg.data.month_splits.val,
    # test_months=[1,2,3,4,5,6,7,8,9,10,11,12], #cfg.data.month_splits.test,
    holdout_months=[],  # =cfg.data.month_splits.holdout,
    cache_dir=os.path.join(
        cfg.data.sdoml.base_directory, cfg.data.sdoml.sub_directory.cache
    ),
    min_date=cfg.data.min_date,
    max_date=cfg.data.max_date,
    num_frames=cfg.data.num_frames,
    drop_frame_dim=False,
)
data_module.setup()

[* CACHE SYSTEM *] Found cached index data in /mnt/sdoml/cache/aligndata_AIA_FULL_12min.csv.
[* CACHE SYSTEM *] Found cached normalization data in /mnt/sdoml/cache/normalizations_AIA_FULL_12min.json.
[* CACHE SYSTEM *] Found cached HMI mask data in /mnt/sdoml/cache/hmi_mask_512x512.npy.


In [6]:
# model = NVAE.load_from_checkpoint('/home/walsh/SDO-FM/outputs/2024-07-02/13-46-18/sdofm/ds441v0c/checkpoints/epoch=5-step=86538.ckpt', map_location=torch.device('cpu'))
model = MAE.load_from_checkpoint(
    "/home/walsh/SDO-FM/notebooks/artifacts/model-q1l4qkjj:v4/model.ckpt",
    map_location=torch.device("cpu"),
)

In [30]:
dl = data_module.test_dataloader()
batch = next(iter(dl))

In [7]:
len(dl)

2907

In [8]:
batch["image_stack"].shape

torch.Size([2, 12, 512, 512])

In [18]:
list(batch["timestamps"][0])

['2011-11-01 00:00:00', '2011-11-01 00:12:00']

In [33]:
x.shape

torch.Size([2, 9, 1, 512, 512])

In [34]:
# z = model.encode(torch.tensor(batch['image_stack']))
# x = model.autoencoder.patch_embed(torch.tensor(batch['image_stack']))
# z, mask, ids_restore = model.forward_encoder(x, mask_ratio=cfg.model.mae.masking_ratio)
x = torch.tensor(batch["image_stack"])
latents, mask, ids = model.autoencoder.forward_encoder(x, mask_ratio=0.5)

/var/tmp/ipykernel_46232/1582853829.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(batch["image_stack"])


In [35]:
latents.shape

torch.Size([2, 513, 2048])

In [37]:
32 * 32

1024

In [20]:
z.detach().cpu().numpy()[0]

array([-0.08459403, -0.79256976,  1.5746768 , ..., -2.729113  ,
       -4.427163  , -3.9137645 ], dtype=float32)

In [22]:
datetime.datetime.now().strftime("%Y_%m_%d_%H%M")

'2024_07_03_23:31'

In [14]:
model.z_dim.shape[0]

21504

In [19]:
model.z_shapes

tensor([[  1.,  32.,  32.],
        [  1.,  64.,  64.],
        [  1., 128., 128.]])

In [23]:
import wandb

run = wandb.init(project="sdofm", id="q1l4qkjj")
artifact = run.use_artifact("fdlx/sdofm/model-q1l4qkjj:v4")
artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dead-water (fdlx). Use `wandb login --relogin` to force relogin


In [26]:
artifact = run.use_artifact("fdlx/sdofm/model-q1l4qkjj:v4")
artifact.download()
path = artifact.get_path("fdlx/sdofm/model-q1l4qkjj:v4")

wandb: Downloading large artifact model-q1l4qkjj:v4, 7294.21MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:57.7


TypeError: Artifact.get_path() missing 1 required positional argument: 'name'

In [37]:
artifact.get_path(name="model.ckpt")

ArtifactManifestEntry(path='model.ckpt', digest='83PI6BY4kSwFK3nyIxP+Hg==', birth_artifact_id='QXJ0aWZhY3Q6OTM2ODQwOTQy', size=7648532105, skip_cache=False)

In [43]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

21410496

NVAE = 21410496 = 21.4M trainable

In [45]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

636477696

MAE = 636477696 = 636M trainable

In [46]:
636 / 21.4

29.719626168224302

In [47]:
21000 / 2048

10.25390625

In [2]:
1025 * 128

131200